# Траектории студентов

Загрузка библиотек

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math

Загрузка файла

In [2]:
def load_data(filename):
    df = pd.read_excel(filename)
    return df

In [3]:
filename = '/content/drive/MyDrive/Colab Notebooks/edu_results.xlsx'

data = load_data(filename)

Удаляем точки в данных и пробелы. Приводим все значения к одному регистру

In [4]:
new_data = data.copy()

In [5]:
# Нужно использовать не просто replace, а str.replace!
# Удалить внутренности скобок. Скобки приходится экранировать с помощью \, иначе они считаются частью регулярного выражения.
new_data['subj'] = new_data['subj'].str.replace('\(.*\)', '', regex=True)

In [6]:
new_data['subj'] = new_data['subj'].str.replace('МДК', '', regex=True)

In [7]:
# Удалить цифры
new_data['subj'] = new_data['subj'].str.replace('\d+', '', regex=True)

In [8]:
# Удаление точек
new_data['subj'] = new_data['subj'].str.replace('\.', '', regex=True)

In [9]:
# Удаление пробелов в начале и конце каждого значения
new_data['subj'] = new_data['subj'].str.strip()

In [10]:
# Преведение к нижнем регистру
# Тут тоже через str.lower!
# Добавил присвоение
new_data['subj'] = new_data['subj'].str.lower()

In [11]:
del new_data['filial']

In [12]:
del new_data['sttext']

In [13]:
del new_data['formobtext']

In [14]:
del new_data['grouptext']

In [15]:
del new_data['yearin']

Преобразование предметов в колонки для анализа оценок по предметам. Если предмета нет, то -1

In [16]:
new_data['mark'] = new_data['mark'].astype('str').str.replace('0', '1', regex=True).astype('int')

In [17]:
# Замена оценок, которые больше 5, на 1
new_data[new_data['mark'] > 5] = 1

In [18]:
# One-hot-encoding столбца subj
new_data = new_data.join(pd.get_dummies(new_data.subj, prefix='subj'))

In [19]:
# Размер таблицы после one-hot-encoding
new_data.shape

(661764, 1565)

In [20]:
# Умножение Названий предметов на оценку

In [21]:
cols = new_data.columns.str.startswith('subj_')
print(cols)
print(len(cols))

[False False False ...  True  True  True]
1565


In [22]:
from tqdm import tqdm

for c in tqdm(new_data.columns[cols]):
    new_data[c] = new_data[c].astype('int').multiply(new_data['mark'], axis=0)

100%|██████████| 1561/1561 [03:41<00:00,  7.05it/s]


In [23]:
# Группировка данных по студенту/семестру
#new_data1 = new_data.groupby(['hash', 'sem']).sum()

In [24]:
#!python -m pip install 'fsspec>=0.3.3'

In [25]:
import dask.dataframe as dd
new_data_1 = dd.from_pandas(new_data, npartitions=3)

In [26]:
result = new_data_1.groupby(['hash', 'sem']).sum()

In [ ]:
result = result.compute()

In [ ]:
#result.to_csv('/content/drive/MyDrive/Colab Notebooks/test1-*.csv')